In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Dense,Flatten,Dropout,Conv2D,MaxPooling2D,BatchNormalization
from tensorflow.keras.optimizers import Adam
import keras
from keras.models import Sequential
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.metrics import AUC

In [ ]:
train_name=os.listdir('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train')
test_name=os.listdir('../input/rsna-miccai-brain-tumor-radiogenomic-classification/test')

In [ ]:
labels=pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')
labels

In [ ]:
sample=pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv')
sample

In [ ]:
def load_dicom(path):
    dicom=pydicom.read_file(path)
    data=dicom.pixel_array
    data=data-np.min(data)
    if np.max(data) != 0:
        data=data/np.max(data)
    data=(data*255).astype(np.uint8)
    return data

In [ ]:
Img_size = 100
path0='../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/T1wCE/Image-1.dcm'
img1=load_dicom(path0)
img2=cv2.resize(img1,(Img_size,Img_size)) 
print(img1.shape)
print(img2.shape)

In [ ]:
train_dir='../input/rsna-miccai-brain-tumor-radiogenomic-classification/train'

for fld in ['FLAIR','T1w','T1wCE','T2w']:
    print('*'*20, fld)
    trainset=[]
    trainlabel=[]
    fig_path = []
    my_label = []
    for i in tqdm(range(len(labels))):
        idt=labels.iloc[i]['BraTS21ID']
        idt2=('00000'+str(idt))[-5:]
        ii=0
        path_to_img = []
        img_nonzero_pix = []
        path=os.path.join(train_dir,idt2,fld) 
        for im in os.listdir(path):
            ii += 1
            img0=load_dicom(os.path.join(path,im))
            if np.max(img0)!=0:
                path_to_img.append(os.path.join(path,im))
                img_nonzero_pix.append(sum(sum(img0!=0)))
        idx = np.argsort(np.array(img_nonzero_pix))
        for k in idx[-16:]:
            fig_path.append(path_to_img[k])#[67:])
            my_label.append(str(labels.iloc[i]['MGMT_value']))

    if fld=='FLAIR':
        flair_train_df = pd.DataFrame({'image_path':fig_path , 'target':my_label});
        flair_train_df.to_csv('./Flair_df.csv')
        display(flair_train_df.head())
    elif fld=='T1w':
        T1w_train_df = pd.DataFrame({'image_path':fig_path , 'target':my_label});
        T1w_train_df.to_csv('./T1w_df.csv')
        display(T1w_train_df.head())
    elif fld=='T1wCE':
        T1wCE_train_df = pd.DataFrame({'image_path':fig_path , 'target':my_label});
        T1wCE_train_df.to_csv('./T1wCE_df.csv')
        display(T1wCE_train_df.head())
    else:
        T2w_train_df = pd.DataFrame({'image_path':fig_path , 'target':my_label});
        T2w_train_df.to_csv('./T2w_df.csv')
        display(T2w_train_df.head())
        